# Procesando datos en paralelo con Dask

<span style="font-size:90%;">
    https://github.com/arielrossanigo/procesando_datos_en_paralelo_con_dask
</span>





**Ariel Rossanigo**



## Quien soy?

* Ariel Rossanigo
* Profe de Inteligencia Artificial
* Developer, Data Scientist

## Objetivo

* Breve introducción a Dask


In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bokeh.plotting import output_notebook
from data_creation import generate_presentation_data
generate_presentation_data()
output_notebook()
print("It works!")

## Dask

<img src="imgs/collections-schedulers.png" width="600" align="middle">


## Dask

#### Por qué?

* API conocida
* Se puede usar local como en cluster
* Se integra con el ecosistema Python
* Soporta aplicaciones complejas
* Tiene herramientas de diagnostico bastante copadas


## Dask Array

<div style="float: left; margin: 30px;"><img src="imgs/dask-array-black-text.svg" width="600" align="middle"></div>


In [ ]:
import dask.array as da

In [ ]:
x = da.arange(25, chunks=5)
x

In [ ]:
y = x ** 2
y

In [ ]:
y.visualize()

In [ ]:
y.compute()

In [ ]:
y.compute(scheduler='processes')

### Numpy vs Dask


In [ ]:
%%time
np_data = np.random.normal(1, size=(20_000, 20_000))
np_data[::50].mean(axis=0)
print("Se usaron {:.2f} GB de RAM".format(np_data.nbytes / 1024**3))

In [ ]:
%%time
da_data = da.random.normal(1, size=(20_000, 20_000), 
                           chunks=(1_000, 1_000))
da_data[::50].mean(axis=0).compute()

### Diagnostics

In [ ]:
from dask.diagnostics import (Profiler, ResourceProfiler, 
                              visualize, ProgressBar)

ProgressBar().register()

In [ ]:
with Profiler() as prof, ResourceProfiler(dt=0.2) as rprof:
    da_data[::50].mean(axis=0).compute()
visualize([prof, rprof], save=False);

## Dask Bag


In [ ]:
import json
import dask.bag as db

data = db.read_text('db_data/*.json.gz')
data.take(1)

In [ ]:
data = data.map(json.loads)
data.take(1)

In [ ]:
promedio = data.pluck('duration').mean()
promedio.visualize()

In [ ]:
promedio.compute()

## Dask Dataframe

<div style="float: left; margin: 30px;"><img src="imgs/dask-dataframe.svg" width="300" align="middle"></div>


**¿Cuando usarlo?**

* El dataset no entra en memoria
* Aprovechar todos los cores para calculos complejos
* Distribuir operaciones comunes de pandas

**¿Cuando NO usarlo?**

* Los datos entran en memoria
* Los datos no son tabulares
* Se necesita hacer algún calculo no estandar

In [ ]:
import dask.dataframe as dd

df = dd.read_csv('dd_data/*.csv.gz', compression='gzip', blocksize=None)
df

In [ ]:
%%time
promedios = df.groupby('month').duration.mean()
promedios.compute()

In [ ]:
%%time
df_pandas = df.compute()
df_pandas.groupby('month').duration.mean()

## Delayed

Permite la creación de grafos modificando levemente el código Python original

In [ ]:
def inc(x):
    return x + 1

def double(x):
    return x * 2

def add(x, y):
    return x + y

output = []
for x in range(1, 6):
    a = inc(x)
    b = double(x)
    c = add(a, b)
    output.append(c)

total = sum(output)
total

In [ ]:
from dask import delayed

@delayed
def inc(x):
    return x + 1

output = []
for x in range(1, 6):
    a = inc(x)
    b = delayed(double)(x)
    c = delayed(add)(a, b)
    output.append(c)

total = delayed(sum)(output)
total.visualize()

In [ ]:
with Profiler() as prof:
    print(total.compute())
visualize([prof], save=False);

### Gracias! Preguntas?


<div style="float: left;"><img src="imgs/man-qmark.jpg" width="300" align="middle"></div> 

<div>
<div>
  <img src="imgs/gmail-1162901_960_720.png" style="width: 30px; float: left; vertical-align:middle; margin: 0px;">
  <span style="line-height:30px; vertical-align:middle; margin-left: 10px;font-size:100%;">arielrossanigo@gmail.com</span>
</div>
<div>
  <img src="imgs/twitter-312464_960_720.png" style="width: 30px; float: left; vertical-align:middle; margin: 0px;">
  <span style="line-height:30px; vertical-align:middle; margin-left: 10px; font-size:100%;">@arielrossanigo</span>
</div>
<div>
  <img src="imgs/github-154769__340.png" style="width: 30px; float: left; vertical-align:middle; margin: 0px;">
  <span style="line-height:30px; vertical-align:middle; margin-left: 10px; font-size:100%;">https://github.com/arielrossanigo</span>
</div>
<div>
  <img src="imgs/Linkedin_icon.svg" style="width: 30px; float: left; vertical-align:middle; margin: 0px;">
  <span style="line-height:30px; vertical-align:middle; margin-left: 10px; font-size:100%;">https://www.linkedin.com/in/arielrossanigo/</span>
</div>

</div>

